In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

import pandas as pd

In [ ]:
forest_data = gpd.read_file("../Exam/Data_OSM/gis_osm_landuse_a_free_1.shp")

In [ ]:
natural = gpd.read_file("../Exam/Data_OSM/gis_osm_natural_free_1.shp")

In [ ]:
places = gpd.read_file("../Exam/Data_OSM/gis_osm_places_a_free_1.shp")

In [ ]:
water = gpd.read_file("../Exam/Data_OSM/gis_osm_water_a_free_1.shp")

In [ ]:
waterways = gpd.read_file("../Exam/Data_OSM/gis_osm_waterways_free_1.shp")

In [ ]:

forest_data["fclass"].unique()

In [ ]:
fig, ax = plt.subplots(figsize=(25, 25))
forest_data[forest_data['fclass'] == 'forest'].plot(ax=ax, color="green", edgecolor='white', linewidth=0.2)
forest_data[forest_data['fclass'] == 'nature_reserve'].plot(ax=ax, color="yellow", edgecolor='white', linewidth=0.2)
forest_data[forest_data['fclass'] == 'meadow'].plot(ax=ax, color="lightgreen", edgecolor='white', linewidth=0.2)
water[water['fclass'] == 'water'].plot(ax=ax, color="blue", edgecolor='white', linewidth=0.2)
waterways.plot(ax=ax, color="lightblue", linewidth=0.2)

plt.show()

In [ ]:
protected_forest = forest_data[forest_data['fclass'] == 'nature_reserve']

fig, ax = plt.subplots(figsize=(10, 10))
protected_forest.plot(ax=ax, color='green', edgecolor='white', linewidth=0.2)

plt.show()


In [ ]:
# Calculate the total area of Denmark
denmark_area = forest_data.geometry.unary_union.area

# Calculate the area of each class in the forest_data
class_areas = forest_data.groupby('fclass').geometry.apply(lambda x: x.unary_union.area)

# Calculate the percentage area of each class
percentage_areas = (class_areas / denmark_area) * 100

# Print the percentage area of each class
print(percentage_areas)


In [ ]:
import folium

forest_copenhagen = forest_data[forest_data['fclass'] == 'forest']
copenhagen_map = folium.Map(location=[55.6761, 12.5683], zoom_start=12)
folium.GeoJson(forest_copenhagen).add_to(copenhagen_map)


copenhagen_map
